In [2]:
import torch
model, transforms = torch.hub.load('harvard-visionlab/lrm-steering', 'alexnet_lrm3', pretrained=True, steering=True, force_reload=True)


Downloading: "https://github.com/harvard-visionlab/lrm-steering/zipball/main" to /n/holylabs/LABS/kempner_fellows/Users/binxuwang/torch_cache/hub/main.zip


==> Loading weights for alexnet_lrm3, hash_id=63ab1b3b06
https://s3.us-east-1.wasabisys.com/visionlab-projects/dnn_feedback_dev/logs/set15/set15_alexnet_torchvision_imagenet1k_lrm_3back_2steps/28453e80-c5e5-4d76-bc81-99c5fade39ff/set15_alexnet_torchvision_imagenet1k_lrm_3back_2steps_final_weights-63ab1b3b06.pth
local_filename: /n/holylabs/LABS/kempner_fellows/Users/binxuwang/torch_cache/hub/set15_alexnet_torchvision_imagenet1k_lrm_3back_2steps_final_weights-63ab1b3b06.pth
<All keys matched successfully>


In [3]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import tqdm
import numpy as np
from os.path import join
import seaborn as sns
import matplotlib.pylab as plt
import torch
import torch.nn.functional as F
from torchvision.transforms import ToPILImage, ToTensor
from torchvision.utils import make_grid
from pytorch_pretrained_biggan import (BigGAN, truncated_noise_sample, one_hot_from_names, save_as_images)
from core.utils.CNN_scorers import TorchScorer
from core.utils.GAN_utils import BigGAN_wrapper, upconvGAN, loadBigGAN
from core.utils.grad_RF_estim import grad_RF_estimate, gradmap2RF_square
from core.utils.layer_hook_utils import get_module_names, layername_dict, register_hook_by_module_names, get_module_name_shapes
from core.utils.layer_hook_utils import featureFetcher, featureFetcher_module, featureFetcher_recurrent
from core.utils.Optimizers import CholeskyCMAES, HessCMAES, ZOHA_Sphere_lr_euclid
from core.utils.Optimizers import label2optimizer


def get_center_pos_and_rf(model, layer, input_size=(3, 227, 227), device="cuda"):
    module_names, module_types, module_spec = get_module_names(model, input_size=input_size, device=device)
    layer_key = [k for k, v in module_names.items() if v == layer][0]
    # FIXME note this may not work when multiple layers have the same name.
    # This will only get the first one. Need to add a check for this.
    feat_outshape = module_spec[layer_key]['outshape']
    if len(feat_outshape) == 3:
        cent_pos = (feat_outshape[1]//2, feat_outshape[2]//2)
    elif len(feat_outshape) == 1:
        cent_pos = None
    else:
        raise ValueError(f"Unknown layer shape {feat_outshape} for layer {layer}")

    # rf Mapping,
    if len(feat_outshape) == 3: # fixit
        print("Computing RF by direct backprop: ")
        gradAmpmap = grad_RF_estimate(model, layer, (slice(None), *cent_pos), input_size=input_size,
                                      device=device, show=False, reps=30, batch=1)
        Xlim, Ylim = gradmap2RF_square(gradAmpmap, absthresh=1E-8, relthresh=0.01, square=True)
        corner = (Xlim[0], Ylim[0])
        imgsize = (Xlim[1] - Xlim[0], Ylim[1] - Ylim[0])
    elif len(feat_outshape) == 1:
        imgsize = input_size[-2:]
        corner = (0, 0)
        Xlim = (corner[0], corner[0] + imgsize[0])
        Ylim = (corner[1], corner[1] + imgsize[1])
    else:
        raise ValueError(f"Unknown layer shape {feat_outshape} for layer {layer}")

    return cent_pos, corner, imgsize, Xlim, Ylim


In [10]:
layername_map = {".feedforward.features.Conv2d0": "conv1",
".feedforward.features.ReLU1": "conv1_relu",
".feedforward.features.MaxPool2d2": "pool1",
".feedforward.features.Conv2d3": "conv2",
".feedforward.features.ReLU4": "conv2_relu",
".feedforward.features.MaxPool2d5": "pool2",
".feedforward.features.Conv2d6": "conv3",
".feedforward.features.ReLU7": "conv3_relu",
".feedforward.features.Conv2d8": "conv4",
".feedforward.features.ReLU9": "conv4_relu",
".feedforward.features.Conv2d10": "conv5",
".feedforward.features.ReLU11": "conv5_relu",
".feedforward.features.MaxPool2d12": "pool5",
".feedforward.AdaptiveAvgPool2davgpool": "avgpool",
".feedforward.classifier.Dropout0": "avgpool_dropout",
".feedforward.classifier.Linear1": "fc6",
".feedforward.classifier.ReLU2": "fc6_relu",
".feedforward.classifier.Dropout3": 'fc6_dropout',
".feedforward.classifier.Linear4": "fc7",
".feedforward.classifier.ReLU5": "fc7_relu",
".feedforward.classifier.Linear6": "fc8",}
layername_inv_map = {v: k for k, v in layername_map.items()}

In [12]:
from easydict import EasyDict as edict
input_size = (3, 224, 224)
model.to("cuda")
args = edict()
args.net = "alexnet_lrm3"
args.layer = layername_inv_map['conv2_relu'] #".feedforward.classifier.ReLU2"
model.forward_passes = 1
cent_pos, corner, imgsize, Xlim, Ylim = get_center_pos_and_rf(model, args.layer,
                                          input_size=input_size, device="cuda")
print("Target setting network %s layer %s, center pos" % (args.net, args.layer), cent_pos)
print("Xlim %s Ylim %s \n imgsize %s corner %s" % (Xlim, Ylim, imgsize, corner))


------------------------------------------------------------------------------
      Layer Id       inshape       outshape           Type                 ReadableStr 
        0        (3, 224, 224) (3, 224, 224)           Input                      Image
        1        (3, 224, 224) (64, 55, 55)          Conv2d  .feedforward.features.Conv2d0
        2        (64, 55, 55) (64, 55, 55)            ReLU  .feedforward.features.ReLU1
        3        (64, 55, 55) (64, 27, 27)       MaxPool2d  .feedforward.features.MaxPool2d2
        4        (64, 27, 27) (192, 27, 27)          Conv2d  .feedforward.features.Conv2d3
        5        (192, 27, 27) (192, 27, 27)            ReLU  .feedforward.features.ReLU4
        6        (192, 27, 27) (192, 13, 13)       MaxPool2d  .feedforward.features.MaxPool2d5
        7        (192, 13, 13) (384, 13, 13)          Conv2d  .feedforward.features.Conv2d6
        8        (384, 13, 13) (384, 13, 13)            ReLU  .feedforward.features.ReLU7
        9      